In [1]:
from keras import backend as K
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import ModelCheckpoint, EarlyStopping

import pandas as pd
import numpy as np
import numpy
import tensorflow as tf
import matplotlib.pyplot as plt
import os
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical

# seed 값 설정
seed = 0
numpy.random.seed(seed)
tf.set_random_seed(seed)


# 데이터 읽어 들이기--- (※1)
#ci_year	ci_term	si_grad	si_ent_gubun	agescale	si_sex	si_last_hak	schoarYn	
#accept_point	avg_logincnt	avg_att_time	avg_att_rate	avg_bbscnt	avg_startlogin	entYn	classcnt	
#avg_te_avg	avg_te_passrate	gradcnt	counsel	stdResult
#features =14
#features =13
#filename="./dataset/20200612_2018_2019.csv"
#filename="./dataset/20200614_v1.csv"
#filename="./dataset/20200717_v1.csv"

features =20
filename="./dataset/20200604_v1.csv"

df_pre=pd.read_csv(filename,encoding="cp949", header=0) # CSV파일을 불러오는 함수를 이용
#df_pre=pd.read_csv('./dataset/20200612_app_point.csv',encoding="cp949", header=0) # CSV파일을 불러오는 함수를 이용





Using TensorFlow backend.
C:\Users\hiranMain\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\hiranMain\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\hiranMain\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\hiranMain\anaconda3\lib\site-packages\tensorflow\python\fra

In [2]:
print("데이터의 총 수", len(df_pre))



데이터의 총 수 58088


In [3]:
df_pre.info()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58088 entries, 0 to 58087
Data columns (total 21 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   ci_year              58088 non-null  int64  
 1   ci_term              58088 non-null  int64  
 2   si_grad              58088 non-null  int64  
 3   si_ent_gubun         58088 non-null  int64  
 4   agescale             58088 non-null  int64  
 5   si_sex               58088 non-null  int64  
 6   si_last_hak          58088 non-null  int64  
 7   schoarYn             58088 non-null  int64  
 8   accept_point         58088 non-null  int64  
 9   avg_logincnt         58088 non-null  int64  
 10  avg_att_time         58088 non-null  int64  
 11  avg_att_rate         58088 non-null  float64
 12  avg_bbscnt           58088 non-null  int64  
 13  avg_startlogin       58088 non-null  int64  
 14  entYn                58088 non-null  int64  
 15  classcnt             58088 non-null 

In [4]:

df_pre.tail()

,ci_year,ci_term,si_grad,si_ent_gubun,agescale,si_sex,si_last_hak,schoarYn,accept_point,avg_logincnt,...,avg_att_rate,avg_bbscnt,avg_startlogin,entYn,classcnt,pre_avg_te_avg,pre_avg_te_passrate,pre_gradcnt,counsel,stdResult
58083,2019,2,4,2,50,1,3,1,70,74,...,100.00,0,30,0,6,3.42,100,1,0,0
58084,2019,2,4,2,50,2,3,1,70,50,...,99.72,0,20,0,8,3.67,100,1,0,0
58085,2019,2,4,2,40,2,3,1,70,63,...,100.00,6,22,0,6,3.33,100,1,0,0
58086,2019,2,4,2,40,2,2,1,70,98,...,100.00,12,47,0,8,3.67,100,1,0,0
58087,2019,2,4,2,30,1,2,1,70,80,...,100.00,0,33,0,7,3.58,100,1,0,0


In [5]:
# 데이터 내부의 기호를 숫자로 변환하기--- (※2)
df = df_pre.sample(frac=1)


dataset = df.values
X = dataset[:,0:features]
Y = dataset[:,features]

# 학습 전용과 테스트 전용 데이터로 나누기 --- (※3)
#테스트, 검증데이터 분할 7:3 
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=seed)

#데이터 정규화
X_train, X_test = X_train / 255, X_test / 255

from keras.layers import Dense, Dropout, Activation
# 모델 설정
model = Sequential()
model.add(Dense(28,  input_dim=features, activation='relu'))
model.add(Dense(15, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(3, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

#모델 컴파일
model.compile(loss='binary_crossentropy',
#model.compile(loss='mean_squared_error',
           optimizer='adam',
           # metrics=['accuracy'])
           metrics=['accuracy',recall_m,precision_m,f1_m])

# 학습 자동 중단 설정 성능이 개선되지 않는 epch 100번까지 수용
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=100)



# 모델 저장 폴더 설정
MODEL_DIR = './model/'
if not os.path.exists(MODEL_DIR):
   os.mkdir(MODEL_DIR)

# 모델 저장 조건 설정
modelpath="./model/{epoch:02d}-{val_loss:.4f}.hdf5"
checkpointer = ModelCheckpoint(filepath=modelpath, monitor='val_loss', verbose=1, save_best_only=True)



# 모델 실행 및 저장
history = model.fit(X_train, Y_train, validation_split=0.20, epochs=500, batch_size=100, 
                    callbacks=[early_stopping_callback, checkpointer])

# 결과 출력
print("\n epoches-500, bat_size=100")
print("\n 학습중단 + 모델 성능개선 : early_stopping_callback:")
print("\n 예측정확도: %.4f" % (model.evaluate(X_test, Y_test)[1]))

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Train on 32528 samples, validate on 8133 samples
Epoch 1/500
32528/32528 [==============================] - 0s 15us/step - loss: 0.6247 - accuracy: 0.6679 - recall_m: 0.9151 - precision_m: 0.6512 - f1_m: 0.7569 - val_loss: 0.6085 - val_accuracy: 0.7001 - val_recall_m: 1.0000 - val_precision_m: 0.7003 - val_f1_m: 0.8230

Epoch 00001: val_loss improved from inf to 0.60851, saving model to ./model/01-0.6085.hdf5
Epoch 2/500
32528/32528 [==============================] - 0s 9us/step - loss: 0.6004 - accuracy: 0.7035 - recall_m: 0.9995 - precision_m: 0.7032 - f1_m: 0.8247 - val_loss: 0.5967 - val_accuracy: 0.7033 - val_recall_m: 0.9880 - val_precision_m: 0.7062 - val_f1_m: 0.8229

Epoch 00002: val_loss improved from 0.60851 to 0.59668, saving model to ./model/02-0.5967.hdf5
Epoch 3/500
32528/32528 [==============================] - 0s 9us/step - loss: 0.5893 - accuracy: 0.7052 - recall_m: 0.9875 -

32528/32528 [==============================] - 0s 9us/step - loss: 0.4535 - accuracy: 0.7815 - recall_m: 0.8675 - precision_m: 0.8352 - f1_m: 0.8472 - val_loss: 0.4444 - val_accuracy: 0.7800 - val_recall_m: 0.8929 - val_precision_m: 0.8121 - val_f1_m: 0.8496

Epoch 00022: val_loss improved from 0.45175 to 0.44437, saving model to ./model/22-0.4444.hdf5
Epoch 23/500
32528/32528 [==============================] - 0s 9us/step - loss: 0.4527 - accuracy: 0.7787 - recall_m: 0.8636 - precision_m: 0.8345 - f1_m: 0.8447 - val_loss: 0.4510 - val_accuracy: 0.7575 - val_recall_m: 0.9169 - val_precision_m: 0.7775 - val_f1_m: 0.8406

Epoch 00023: val_loss did not improve from 0.44437
Epoch 24/500
32528/32528 [==============================] - 0s 9us/step - loss: 0.4483 - accuracy: 0.7819 - recall_m: 0.8622 - precision_m: 0.8384 - f1_m: 0.8467 - val_loss: 0.4474 - val_accuracy: 0.7909 - val_recall_m: 0.8052 - val_precision_m: 0.8858 - val_f1_m: 0.8427

Epoch 00024: val_loss did not improve from 0.444

32528/32528 [==============================] - 0s 9us/step - loss: 0.4348 - accuracy: 0.7940 - recall_m: 0.8441 - precision_m: 0.8640 - f1_m: 0.8511 - val_loss: 0.4274 - val_accuracy: 0.7974 - val_recall_m: 0.8680 - val_precision_m: 0.8462 - val_f1_m: 0.8561

Epoch 00046: val_loss did not improve from 0.42668
Epoch 47/500
32528/32528 [==============================] - 0s 9us/step - loss: 0.4379 - accuracy: 0.7915 - recall_m: 0.8452 - precision_m: 0.8624 - f1_m: 0.8497 - val_loss: 0.4260 - val_accuracy: 0.8015 - val_recall_m: 0.8566 - val_precision_m: 0.8595 - val_f1_m: 0.8571

Epoch 00047: val_loss improved from 0.42668 to 0.42602, saving model to ./model/47-0.4260.hdf5
Epoch 48/500
32528/32528 [==============================] - 0s 8us/step - loss: 0.4362 - accuracy: 0.7939 - recall_m: 0.8484 - precision_m: 0.8625 - f1_m: 0.8519 - val_loss: 0.4415 - val_accuracy: 0.7936 - val_recall_m: 0.7852 - val_precision_m: 0.9078 - val_f1_m: 0.8411

Epoch 00048: val_loss did not improve from 0.426


Epoch 00070: val_loss did not improve from 0.42221
Epoch 71/500
32528/32528 [==============================] - 0s 9us/step - loss: 0.4326 - accuracy: 0.7956 - recall_m: 0.8469 - precision_m: 0.8649 - f1_m: 0.8527 - val_loss: 0.4274 - val_accuracy: 0.7933 - val_recall_m: 0.8883 - val_precision_m: 0.8284 - val_f1_m: 0.8566

Epoch 00071: val_loss did not improve from 0.42221
Epoch 72/500
32528/32528 [==============================] - 0s 9us/step - loss: 0.4284 - accuracy: 0.8020 - recall_m: 0.8472 - precision_m: 0.8701 - f1_m: 0.8564 - val_loss: 0.4225 - val_accuracy: 0.8078 - val_recall_m: 0.8289 - val_precision_m: 0.8889 - val_f1_m: 0.8571

Epoch 00072: val_loss did not improve from 0.42221
Epoch 73/500
32528/32528 [==============================] - 0s 10us/step - loss: 0.4309 - accuracy: 0.7980 - recall_m: 0.8455 - precision_m: 0.8675 - f1_m: 0.8534 - val_loss: 0.4219 - val_accuracy: 0.8109 - val_recall_m: 0.8379 - val_precision_m: 0.8859 - val_f1_m: 0.8604

Epoch 00073: val_loss impr

32528/32528 [==============================] - 0s 9us/step - loss: 0.4269 - accuracy: 0.7997 - recall_m: 0.8460 - precision_m: 0.8699 - f1_m: 0.8547 - val_loss: 0.4262 - val_accuracy: 0.7952 - val_recall_m: 0.9019 - val_precision_m: 0.8225 - val_f1_m: 0.8597

Epoch 00095: val_loss did not improve from 0.41475
Epoch 96/500
32528/32528 [==============================] - 0s 9us/step - loss: 0.4220 - accuracy: 0.8076 - recall_m: 0.8520 - precision_m: 0.8740 - f1_m: 0.8608 - val_loss: 0.4131 - val_accuracy: 0.8126 - val_recall_m: 0.8561 - val_precision_m: 0.8740 - val_f1_m: 0.8643

Epoch 00096: val_loss improved from 0.41475 to 0.41308, saving model to ./model/96-0.4131.hdf5
Epoch 97/500
32528/32528 [==============================] - 0s 9us/step - loss: 0.4208 - accuracy: 0.8060 - recall_m: 0.8488 - precision_m: 0.8742 - f1_m: 0.8594 - val_loss: 0.4359 - val_accuracy: 0.7915 - val_recall_m: 0.7774 - val_precision_m: 0.9124 - val_f1_m: 0.8385

Epoch 00097: val_loss did not improve from 0.413

...

32528/32528 [==============================] - 0s 9us/step - loss: 0.3458 - accuracy: 0.8551 - recall_m: 0.9240 - precision_m: 0.8773 - f1_m: 0.8991 - val_loss: 0.3571 - val_accuracy: 0.8545 - val_recall_m: 0.9060 - val_precision_m: 0.8882 - val_f1_m: 0.8965

Epoch 00391: val_loss did not improve from 0.33733
Epoch 392/500
32528/32528 [==============================] - 0s 8us/step - loss: 0.3500 - accuracy: 0.8533 - recall_m: 0.9214 - precision_m: 0.8769 - f1_m: 0.8974 - val_loss: 0.3435 - val_accuracy: 0.8565 - val_recall_m: 0.9185 - val_precision_m: 0.8812 - val_f1_m: 0.8988

Epoch 00392: val_loss did not improve from 0.33733
Epoch 393/500
32528/32528 [==============================] - 0s 8us/step - loss: 0.3460 - accuracy: 0.8545 - recall_m: 0.9246 - precision_m: 0.8761 - f1_m: 0.8988 - val_loss: 0.3413 - val_accuracy: 0.8582 - val_recall_m: 0.9373 - val_precision_m: 0.8700 - val_f1_m: 0.9017

Epoch 00393: val_loss did not improve from 0.33733
Epoch 394/500
32528/32528 [============


Epoch 00415: val_loss did not improve from 0.33475
Epoch 416/500
32528/32528 [==============================] - 0s 8us/step - loss: 0.3435 - accuracy: 0.8545 - recall_m: 0.9260 - precision_m: 0.8754 - f1_m: 0.8989 - val_loss: 0.3354 - val_accuracy: 0.8611 - val_recall_m: 0.9438 - val_precision_m: 0.8689 - val_f1_m: 0.9043

Epoch 00416: val_loss did not improve from 0.33475
Epoch 417/500
32528/32528 [==============================] - 0s 8us/step - loss: 0.3431 - accuracy: 0.8559 - recall_m: 0.9254 - precision_m: 0.8771 - f1_m: 0.8996 - val_loss: 0.3407 - val_accuracy: 0.8586 - val_recall_m: 0.9187 - val_precision_m: 0.8839 - val_f1_m: 0.9003

Epoch 00417: val_loss did not improve from 0.33475
Epoch 418/500
32528/32528 [==============================] - 0s 9us/step - loss: 0.3443 - accuracy: 0.8541 - recall_m: 0.9245 - precision_m: 0.8757 - f1_m: 0.8984 - val_loss: 0.3396 - val_accuracy: 0.8561 - val_recall_m: 0.9371 - val_precision_m: 0.8676 - val_f1_m: 0.9005

Epoch 00418: val_loss di


Epoch 00440: val_loss did not improve from 0.33074
Epoch 441/500
32528/32528 [==============================] - 0s 10us/step - loss: 0.3422 - accuracy: 0.8564 - recall_m: 0.9280 - precision_m: 0.8757 - f1_m: 0.9002 - val_loss: 0.3326 - val_accuracy: 0.8593 - val_recall_m: 0.9285 - val_precision_m: 0.8773 - val_f1_m: 0.9016

Epoch 00441: val_loss did not improve from 0.33074
Epoch 442/500
32528/32528 [==============================] - 0s 11us/step - loss: 0.3411 - accuracy: 0.8569 - recall_m: 0.9282 - precision_m: 0.8762 - f1_m: 0.9006 - val_loss: 0.3445 - val_accuracy: 0.8571 - val_recall_m: 0.9472 - val_precision_m: 0.8620 - val_f1_m: 0.9020

Epoch 00442: val_loss did not improve from 0.33074
Epoch 443/500
32528/32528 [==============================] - 0s 9us/step - loss: 0.3372 - accuracy: 0.8580 - recall_m: 0.9311 - precision_m: 0.8752 - f1_m: 0.9014 - val_loss: 0.3342 - val_accuracy: 0.8590 - val_recall_m: 0.9164 - val_precision_m: 0.8856 - val_f1_m: 0.9002

Epoch 00443: val_loss 

32528/32528 [==============================] - 0s 10us/step - loss: 0.3364 - accuracy: 0.8573 - recall_m: 0.9318 - precision_m: 0.8739 - f1_m: 0.9011 - val_loss: 0.3330 - val_accuracy: 0.8590 - val_recall_m: 0.9268 - val_precision_m: 0.8780 - val_f1_m: 0.9012

Epoch 00465: val_loss did not improve from 0.32763
Epoch 466/500
32528/32528 [==============================] - 0s 10us/step - loss: 0.3400 - accuracy: 0.8559 - recall_m: 0.9298 - precision_m: 0.8740 - f1_m: 0.9000 - val_loss: 0.3432 - val_accuracy: 0.8531 - val_recall_m: 0.9037 - val_precision_m: 0.8879 - val_f1_m: 0.8952

Epoch 00466: val_loss did not improve from 0.32763
Epoch 467/500
32528/32528 [==============================] - 0s 8us/step - loss: 0.3384 - accuracy: 0.8568 - recall_m: 0.9316 - precision_m: 0.8736 - f1_m: 0.9009 - val_loss: 0.3360 - val_accuracy: 0.8556 - val_recall_m: 0.9027 - val_precision_m: 0.8920 - val_f1_m: 0.8967

Epoch 00467: val_loss did not improve from 0.32763
Epoch 468/500
32528/32528 [==========

32528/32528 [==============================] - 0s 9us/step - loss: 0.3342 - accuracy: 0.8589 - recall_m: 0.9345 - precision_m: 0.8741 - f1_m: 0.9025 - val_loss: 0.3298 - val_accuracy: 0.8607 - val_recall_m: 0.9346 - val_precision_m: 0.8745 - val_f1_m: 0.9030

Epoch 00490: val_loss did not improve from 0.32583
Epoch 491/500
32528/32528 [==============================] - 0s 8us/step - loss: 0.3362 - accuracy: 0.8578 - recall_m: 0.9325 - precision_m: 0.8741 - f1_m: 0.9015 - val_loss: 0.3324 - val_accuracy: 0.8612 - val_recall_m: 0.9410 - val_precision_m: 0.8706 - val_f1_m: 0.9038

Epoch 00491: val_loss did not improve from 0.32583
Epoch 492/500
32528/32528 [==============================] - 0s 8us/step - loss: 0.3371 - accuracy: 0.8577 - recall_m: 0.9332 - precision_m: 0.8737 - f1_m: 0.9016 - val_loss: 0.3362 - val_accuracy: 0.8582 - val_recall_m: 0.9159 - val_precision_m: 0.8852 - val_f1_m: 0.8997

Epoch 00492: val_loss did not improve from 0.32583
Epoch 493/500
32528/32528 [============

In [7]:
#테스트 데이터 검증
loss, accuracy, recall,precision,f1_socre  = model.evaluate(X_test, Y_test)
#accuracy  = model.evaluate(X_test, Y_test)

print("\n accuracy, precision, recall, f1_score")
print("\n  %.4f, %.4f, %4f, %.4f" % (accuracy, precision ,recall, f1_socre))

#print("\n accuracy")
#print("\n  %.4f % (accuracy)")

17427/17427 [==============================] - 0s 10us/step

 accuracy, precision, recall, f1_score

  0.8557, 0.8766, 0.923183, 0.8973
